#Import libraies and packages

In [1]:
pip install ray torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import pandas as pd
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Model

In [12]:
import tensorflow as tf

def ConvNet(config, len_classes=2):
    input = tf.keras.layers.Input(shape=(400, 1))
    x = input
    # for i in range(config['layers']):
    x = tf.keras.layers.Conv1D(filters=config['conv_block1_filters'], kernel_size=3, strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv1D(filters=config['conv_block2_filters'], kernel_size=3, strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)


    x = tf.keras.layers.Conv1D(filters=config['conv_block3_filters'], kernel_size=3, strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)


    x = tf.keras.layers.Conv1D(filters=config['conv_block4_filters'], kernel_size=2, strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv1D(filters=config['conv_block5_filters'], kernel_size=1, strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    if config['fc_layer_type'] == 'dense':
        if config['pool_type'] == 'max':
            x = tf.keras.layers.GlobalMaxPooling1D()(x)
        else:
            x = tf.keras.layers.GlobalAveragePooling1D()(x)

        # Fully connected layer 1
        x = tf.keras.layers.Dense(units=config['fc1_units'])(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)

        # Fully connected layer 2
        x = tf.keras.layers.Dense(units=len_classes)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        predictions = tf.keras.layers.Dense(1,tf.keras.layers.Activation('sigmoid'))(x)

    else:
        # Fully connected layer 1
        x = tf.keras.layers.Conv1D(filters=config['fc1_units'], kernel_size=1, strides=1)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)


        # Fully connected layer 2
        x = tf.keras.layers.Conv1D(filters=len_classes, kernel_size=1, strides=1)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        if config['pool_type'] == 'max':
            x = tf.keras.layers.GlobalMaxPooling1D()(x)
        else:
            x = tf.keras.layers.GlobalAveragePooling1D()(x)
        
        predictions = tf.keras.layers.Dense(1,tf.keras.layers.Activation('sigmoid'))(x)

    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model
   

## Metrics

In [6]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Training function

In [7]:
from sklearn import preprocessing

def train_mnist( config):
  path ='/content/drive/MyDrive/UPM/Clinical_data_and_RNA_total_Features_PFS.csv'
  data_frame = pd.read_csv(path)
  
  from sklearn.model_selection import train_test_split
  X = data_frame[['5S_rRNA', 'AADACL2-AS1', 'AB015752.3', 'AB019438.66', 'ABHD17AP7', 'AC000077.2', 'AC000111.3', 'AC000111.5', 'AC000124.1', 'AC002368.4', 'AC002472.11', 'AC002486.3', 'AC002542.2', 'AC003084.2', 'AC003682.16', 'AC003682.17', 'AC003986.5', 'AC004006.2', 'AC004012.1', 'AC004014.3', 'AC004066.3', 'AC004112.4', 'AC004158.3', 'AC004221.2', 'AC004449.6', 'AC004510.3', 'AC004549.6', 'AC004862.6', 'AC004870.3', 'AC004941.3', 'AC004947.2', 'AC005008.2', 'AC005094.2', 'AC005178.1', 'AC005197.2', 'AC005235.1', 'AC005307.1', 'AC005355.1', 'AC005387.3', 'AC005392.13', 'AC005498.4', 'AC005518.2', 'AC005550.5', 'AC005597.1', 'AC005624.2', 'AC005722.4', 'AC005740.5', 'AC006003.3', 'AC006007.1', 'AC006041.1', 'AC006116.14', 'AC006130.4', 'AC006153.3', 'AC006296.2', 'AC006372.4', 'AC006372.5', 'AC006372.6', 'AC006373.1', 'AC006427.2', 'AC006499.1', 'AC006499.3', 'AC006499.5', 'AC006499.6', 'AC006509.4', 'AC006960.7', 'AC006987.5', 'AC007128.1', 'AC007131.2', 'AC007163.6', 'AC007193.9', 'AC007249.3', 'AC007253.1', 'AC007292.7', 'AC007322.1', 'AC007349.7', 'AC007391.2', 'AC007403.3', 'AC007463.2', 'AC007557.4', 'AC007563.3', 'AC007563.5', 'AC007568.1', 'AC007679.3', 'AC007682.1', 'AC007740.1', 'AC007742.3', 'AC007796.1', 'AC007879.1', 'AC007879.3', 'AC008067.2', 'AC008074.4', 'AC008154.4', 'AC008278.2', 'AC008281.1', 'AC008746.10', 'AC008746.9', 'AC008834.2', 'AC008940.1', 'AC009161.1', 'AC009237.16', 'AC009237.17', 'AC009238.7', 'AC009238.8', 'AC009310.1', 'AC009312.1', 'AC009313.1', 'AC009410.1', 'AC009477.7', 'AC009487.4', 'AC009487.5', 'AC009541.1', 'AC009970.1', 'AC010096.2', 'AC010141.1', 'AC010518.3', 'AC010729.1', 'AC010729.2', 'AC010731.2', 'AC010731.6', 'AC010884.1', 'AC010907.5', 'AC010967.1', 'AC010967.2', 'AC010974.3', 'AC010983.1', 'AC010987.6', 'AC011196.3', 'AC011243.1', 'AC011286.1', 'AC011343.1', 'AC011516.2', 'AC011524.3', 'AC011648.1', 'AC011742.5', 'AC012074.2', 'AC012075.2', 'AC012322.1', 'AC012354.6', 'AC012363.13', 'AC012456.4', 'AC012506.1', 'AC012506.2', 'AC012506.3', 'AC012506.4', 'AC012507.3', 'AC012507.4', 'AC012513.6', 'AC012668.1', 'AC013402.5', 'AC013429.4', 'AC013448.2', 'AC013480.2', 'AC013733.3', 'AC015936.3', 'AC016694.2', 'AC016735.1', 'AC016745.3', 'AC016831.3', 'AC016831.5', 'AC016912.3', 'AC017019.1', 'AC017048.2', 'AC018359.1', 'AC018643.4', 'AC018685.1', 'AC018717.1', 'AC018799.1', 'AC018804.3', 'AC018880.1', 'AC018880.2', 'AC018892.3', 'AC019064.1', 'AC019118.4', 'AC020595.1', 'AC020743.2', 'AC020915.2', 'AC022201.4', 'AC023115.4', 'AC023128.1', 'AC024082.4', 'AC024704.2', 'AC026202.5', 'AC026954.6', 'AC027124.2', 'AC034228.2', 'AC060834.3', 'AC062016.1', 'AC062022.1', 'AC063976.1', 'AC063976.3', 'AC064834.1', 'AC064834.2', 'AC067945.2', 'AC067956.1', 'AC068057.1', 'AC068490.1', 'AC068492.1', 'AC068535.3', 'AC068831.3', 'AC069155.1', 'AC073218.1', 'AC073464.6', 'AC073641.2', 'AC074338.4', 'AC078842.3', 'AC078882.1', 'AC079112.1', 'AC079117.1', 'AC079145.4', 'AC079154.1', 'AC079325.6', 'AC079586.1', 'AC079610.1', 'AC079776.1', 'AC079987.2', 'AC083864.3', 'AC083939.1', 'AC090018.1', 'AC090952.5', 'AC090957.2', 'AC091133.1', 'AC091729.8', 'AC091814.2', 'AC092155.4', 'AC092159.3', 'AC092168.3', 'AC092566.1', 'AC092570.1', 'AC092570.2', 'AC092578.1', 'AC092601.1', 'AC092635.1', 'AC092657.2', 'AC092933.4', 'AC093063.2', 'AC093084.1', 'AC093106.4', 'AC093326.1', 'AC093381.2', 'AC093590.1', 'AC093642.6', 'AC096582.9', 'AC096669.2', 'AC096732.2', 'AC097468.7', 'AC097713.3', 'AC097713.4', 'AC098592.6', 'AC098828.3', 'AC098872.3', 'AC098973.1', 'AC099344.2', 'AC099344.3', 'AC099552.3', 'AC102948.2', 'AC103563.5', 'AC103563.7', 'AC103563.8', 'AC104076.3', 'AC104306.4', 'AC104777.2', 'AC104777.4', 'AC107083.1', 'AC108032.1', 'AC108066.1', 'AC109589.1', 'AC110620.1', 'AC110754.3', 'AC112198.1', 'AC112220.2', 'AC112518.3', 'AC113167.1', 'AC113167.2', 'AC113607.2', 'AC114752.3', 'AC114814.2', 'AC114877.3', 'AC118653.2', 'AC123900.2', 'AC124057.5', 'AC125421.1', 'AC128709.2', 'AC128709.3', 'AC128709.4', 'AC133633.1', 'AC133680.1', 'AC133785.1', 'AC140481.8', 'AC142119.1', 'AC142293.3', 'AC144450.1', 'AC144833.1', 'ADI1P2', 'AE000661.36', 'AF186192.6', 'AHCYP3', 'AL133249.1', 'AP000345.4', 'AP000356.2', 'AP000619.5', 'AP000648.6', 'AP000997.2', 'AP006285.7', 'ATP5F1P4', 'ATP5G1P8', 'BAATP1', 'BOK-AS1', 'BRI3BPP1', 'BSN-AS1', 'BSN-AS2', 'CASC16', 'CDCA4P3', 'CDKN2AIPNLP2', 'CICP1', 'CLUHP4', 'CNN2P2', 'CTA-392E5.1', 'CTB-138E5.1', 'CTB-164N12.1', 'CTB-49A3.1', 'CTC-304I17.5', 'CTC-339O9.2', 'CTC-501O10.1', 'CTD-2130F23.2', 'CTD-2269E23.3', 'CTD-2308B18.1', 'CTD-2319I12.5', 'CTD-2555O16.1', 'CTD-3187F8.11', 'CYCSP26', 'CYP2T3P', 'DDX43P2', 'DLG3-AS1', 'DPRXP6', 'GLUD1P4', 'GLULP6', 'GRAMD4P7', 'GS1-114I9.1', 'GS1-251I9.2', 'GTSCR1', 'HNRNPH3P1', 'IFT74-AS1', 'IGKV7-3', 'KRT8P6', 'KRTAP20-3', 'LINC00309', 'LINC00364', 'LINC00391', 'LINC00466', 'LINC00838', 'LINC00856', 'LINC00864', 'LINC00879', 'LINC01326', 'LINC01524', 'LRRC34P1', 'MARCKSL1P2', 'MIR1273F', 'MIR4290HG', 'MRLN', 'MTATP6P23', 'MTCO2P16', 'NHEG1', 'NOS2P4', 'OR10B1P', 'OR13Z2P', 'OR2AD1P', 'OR2B8P', 'OR5BN2P', 'OSBPL9P1', 'PDX1-AS1', 'PGBD4P5', 'RN7SKP238', 'RNA5SP205', 'RNF212', 'RNU6-1090P', 'RP1-151B14.9', 'RP11-100K18.1', 'RP11-107N7.1', 'RP11-123J14.1', 'RP11-203B7.2', 'RP11-2H8.3', 'RP11-307O10.1', 'RP11-359G22.2', 'RP11-361A23.3', 'RP11-378I6.1', 'RP11-430C1.1', 'RP11-434D11.4', 'RP11-438B23.2', 'RP11-520D19.2', 'RP11-661A12.12', 'RP11-846C15.2', 'RP11-97F8.1', 'RP13-631K18.2', 'RP3-406P24.4', 'RP4-581O6.1', 'RP5-998G20.1', 'RPL35AP', 'XXyac-YR12DB5.1', 'YBX1P9']].astype(float)
  Y=[]
  for i in range (len(data_frame)):
      if data_frame.Target[i]=='NR': # If PFS is lower than 3 months, I will consider it as NonResponder (NR)
          Y.append(0)
      else:
          Y.append(1)# If PFS is over 3 months, I will consider it as Responder (R)
  scaler = preprocessing.MinMaxScaler()
  names = X.columns
  d = scaler.fit_transform(X)
  X = pd.DataFrame(d, columns=names)
  XTrain, XTest, yTrain, yTest = train_test_split(X, Y, test_size=0.20, random_state=125)
  # Convert sets to arrays
  XTrain = XTrain.values
  XTest = XTest.values
  # It is mandatory to transform Y list into array for trainning the model
  yTrain=np.array(yTrain)
  yTest=np.array(yTest)

  # building the input vector from the 20x20 pixels
  X_train = XTrain.reshape(XTrain.shape[0], 400 , 1)
  X_test = XTest.reshape(XTest.shape[0], 400, 1)
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  # Create model
  model = ConvNet(config)

  # Compile model with losses and metrics
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate =config['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy', f1_m,precision_m, recall_m
                           ])

  # Start model training
  history_m = model.fit(X_train, yTrain,
                      epochs=1,
                      validation_data=(X_test, yTest),
                      )

  history_m = {
  "loss": history_m.history["loss"][0],
  "val_loss": history_m.history["val_loss"][0],
  "accuracy": history_m.history["accuracy"][0],
  "val_accuracy": history_m.history["val_accuracy"][0]
  }
  return history_m

# Bayesian Optimization

In [30]:
pip install hpbandster ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 84 kB/s 
     |████████████████████████████████| 89 kB 7.6 MB/s 
  Created wheel for hpbandster: filename=hpbandster-0.7.4-py3-none-any.whl size=80006 sha256=470ddce526a7f1d4ff7552f9d8abcd5e0c3c1ea609d2a5f4e228fe983ede8828
  Stored in directory: /root/.cache/pip/wheels/d9/88/fc/61ab6b9f386a386839668631c39a6dc3c2fb0ec7000d552faa
Successfully built hpbandster


In [26]:
pip install ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import ConfigSpace as CS
config_space = CS.ConfigurationSpace()

In [9]:
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="lr", choices=[0.0001, 0.001, 0.01, 0.1]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="dropout_rate", choices=[0.1, 0.2, 0.3, 0.4, 0.5]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="conv_block1_filters", choices=[32, 64, 128, 256, 512]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="conv_block2_filters", choices=[32, 64, 128, 256, 512]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="conv_block3_filters", choices=[32, 64, 128, 256, 512]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="conv_block4_filters", choices=[32, 64, 128, 256, 512]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="conv_block5_filters", choices=[32, 64, 128, 256]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="fc_layer_type", choices= ['dense', 'convolution']))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="pool_type", choices= ['max', 'average']))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="fc1_units", choices=[32, 64, 128, 256, 512]))

fc1_units, Type: Categorical, Choices: {32, 64, 128, 256, 512}, Default: 32

In [13]:
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB

algo = TuneBOHB(
    config_space, metric="val_accuracy", mode="max")
bohb = HyperBandForBOHB(
    time_attr="training_iteration",
    metric="val_accuracy",
    mode="max",
    max_t=100)
tune.run(train_mnist, scheduler=bohb, search_alg=algo)

2022-05-24 17:01:15,747	WARNING callback.py:126 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2022-05-24 17:01:15,755	WARNING tune.py:637 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.
2022-05-24 17:01:15,871	INFO trial_runner.py:803 -- starting train_mnist_1f8470da


Trial name,status,loc,conv_block1_filters,conv_block2_filters,conv_block3_filters,conv_block4_filters,conv_block5_filters,dropout_rate,fc1_units,fc_layer_type,lr,pool_type
train_mnist_1f8470da,RUNNING,172.28.0.2:9286,128,256,128,64,256,0.2,128,dense,0.0001,average


Trial name,status,loc,conv_block1_filters,conv_block2_filters,conv_block3_filters,conv_block4_filters,conv_block5_filters,dropout_rate,fc1_units,fc_layer_type,lr,pool_type
train_mnist_1f8470da,RUNNING,172.28.0.2:9286,128,256,128,64,256,0.2,128,dense,0.0001,average


(train_mnist pid=9286) 2022-05-24 17:01:27.325050: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


(train_mnist pid=9286) Model: "model"
(train_mnist pid=9286) _________________________________________________________________
(train_mnist pid=9286)  Layer (type)                Output Shape              Param #   
(train_mnist pid=9286) =================================================================
(train_mnist pid=9286)  input_1 (InputLayer)        [(None, 400, 1)]          0         
(train_mnist pid=9286)                                                                  
(train_mnist pid=9286)  conv1d (Conv1D)             (None, 398, 128)          512       
(train_mnist pid=9286)                                                                  
(train_mnist pid=9286)  batch_normalization (BatchN  (None, 398, 128)         512       
(train_mnist pid=9286)  ormalization)                                                   
(train_mnist pid=9286)                                                                  
(train_mnist pid=9286)  activation (Activation)     (None, 398, 128)    

...

2022-05-24 17:01:49,267	INFO hyperband.py:467 -- Restoring from a previous point in time. Previous=1; Now=1


Result for train_mnist_1f8470da:
  accuracy: 0.5555555820465088
  date: 2022-05-24_17-01-49
  done: false
  episodes_total: 0
  experiment_id: d6e86483659d4663bcfa83e15a865a64
  hostname: bec2c43fd2a7
  iterations_since_restore: 1
  loss: 0.73299241065979
  node_ip: 172.28.0.2
  pid: 9354
  time_since_restore: 12.765836954116821
  time_this_iter_s: 12.765836954116821
  time_total_s: 25.667383909225464
  timestamp: 1653411709
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 1
  trial_id: 1f8470da
  val_accuracy: 0.6756756901741028
  val_loss: 0.6930209398269653
  warmup_time: 0.004525899887084961
  
Result for train_mnist_1f8470da:
  accuracy: 0.5555555820465088
  date: 2022-05-24_17-01-49
  done: true
  episodes_total: 0
  experiment_id: d6e86483659d4663bcfa83e15a865a64
  experiment_tag: 1_conv_block1_filters=128,conv_block2_filters=256,conv_block3_filters=128,conv_block4_filters=64,conv_block5_filters=256,dropout_rate=0.2,fc1_units=128,fc_layer_type=dense,lr=0.0

Trial name,status,loc,conv_block1_filters,conv_block2_filters,conv_block3_filters,conv_block4_filters,conv_block5_filters,dropout_rate,fc1_units,fc_layer_type,lr,pool_type,iter,total time (s),ts,loss,val_loss,accuracy
train_mnist_1f8470da,TERMINATED,172.28.0.2:9354,128,256,128,64,256,0.2,128,dense,0.0001,average,1,25.6674,0,0.732992,0.693021,0.555556


2022-05-24 17:01:49,404	INFO tune.py:702 -- Total run time: 33.69 seconds (33.52 seconds for the tuning loop).


5/5 [==============================] - 5s 493ms/step - loss: 0.7330 - accuracy: 0.5556 - f1_m: 0.5499 - precision_m: 0.5958 - recall_m: 0.5231 - val_loss: 0.6930 - val_accuracy: 0.6757 - val_f1_m: 0.5448 - val_precision_m: 0.5833 - val_recall_m: 0.5238
